In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
from fastai.text import *
from sebastian.eval import eval_ner
from model.data_utils import minibatches, pad_sequences, get_chunks
from model.config import Config
from model.data_utils import CoNLLDataset
from torch.utils.data.sampler import SequentialSampler, RandomSampler, BatchSampler
import torch.nn.utils.rnn as rnn_utils
from UTILS.lstm_v import LSTM_v1

/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/pytorch4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may

In [3]:
config = Config()

In [4]:
train = CoNLLDataset(config.filename_train, config.processing_word,
                         config.processing_tag, config.max_iter)
val = CoNLLDataset(config.filename_dev, config.processing_word,
                         config.processing_tag, config.max_iter)
test = CoNLLDataset(config.filename_test, config.processing_word,
                         config.processing_tag, config.max_iter)

In [49]:
class Minibatch(object):
    def __init__(self, data, minibatch_size):
        self.data = data
        self.minibatch_size = minibatch_size
        self.length = None
        
    def __iter__(self):
        x_batch, y_batch = [], []
        for (x, y) in self.data:
            if len(x_batch) == self.minibatch_size:
                char_ids, word_ids = zip(*x_batch)
                word_ids, sequence_lengths = pad_sequences(word_ids, 0)
                char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
                    nlevels=2)
                lbl_ids, lbl_lengths = pad_sequences(y_batch, 0)
                bs, sl, char = np.array(char_ids).shape
                # expand the seq_lens and pad with ones, and concat with the rest
                # seq_len (bs)
                seq_lens_padded = np.concatenate([np.array(sequence_lengths)[:,np.newaxis], np.ones((bs, sl-1))], axis=-1)
                # seq_len, word_ids, word_length, char_ids
                word_ids = np.concatenate([seq_lens_padded[:,:,None], 
                                           np.array(word_ids)[:,:,None], 
                                           np.array(word_lengths)[:,:,None], 
                                           np.array(char_ids)], axis=-1)
                
                yield T(word_ids), T(lbl_ids).view(-1)
                x_batch, y_batch = [], []

            if type(x[0]) == tuple:
                x = zip(*x)
            x_batch += [x]
            y_batch += [y]

#         if len(x_batch) != 0:
#             char_ids, word_ids = zip(*x_batch)
#             word_ids, sequence_lengths = pad_sequences(word_ids, 0)
#             char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0,
#                 nlevels=2)
#             lbl_ids, lbl_lengths = pad_sequences(y_batch, 0)
#             word_ids = np.concatenate([np.array(word_ids)[:,:,np.newaxis], np.array(char_ids)], axis=-1)
#             yield T(word_ids), T(lbl_ids).view(-1)
    
    def __len__(self):
        if self.length == None:
            self.length = 0
            for _ in self:
                self.length += 1
        return self.length

In [50]:
dir_path = '/home/'
trn_dl = Minibatch(train, 20)
val_dl = Minibatch(val, 20)
test_dl = Minibatch(test, 20)
md = ModelData(dir_path, trn_dl, val_dl, test_dl)

In [51]:
class NER_model(nn.Module):
    def __init__(self, config):
        super(NER_model, self).__init__()
        self.config = config
        self.idx_to_tag = {idx: tag for tag, idx in
                           self.config.vocab_tags.items()}
        self.get_word_embeddings()
        self.get_logits()
        
    def get_word_embeddings(self):
        # get word embeding
        _word_embedding = V(self.config.embeddings, requires_grad=True)
        self.word_embedding = nn.Embedding.from_pretrained(_word_embedding)
        
        # get char embedding
        self._char_embedding = nn.Embedding(self.config.nchars, self.config.dim_char)
        self.char_embedding = LSTM_v1(input_size=self.config.dim_char, hidden_size=self.config.hidden_size_char,
                                     num_layers=1, batch_first=True, # not sure here whether batch is first
                                     bidirectional=True)
        
    def get_logits(self):
        self.rnn = LSTM_v1(input_size=self.config.dim_word+self.config.dim_char*2,
                          hidden_size=self.config.hidden_size_lstm,
                          num_layers=1, batch_first=True, # not sure whether batch is first
                          bidirectional=True)
        self.dropout = nn.Dropout(self.config.dropout if self.train else 0)
        self.linear = nn.Linear(self.config.hidden_size_lstm*2, self.config.ntags)
        
    def forward(self, input):
        seq_lens = input[:,0,0]
        words = input[:, :, 1]
        word_lens = input[:,:,2]
        char = input[:, :, 3:]
        
        bs, sl, _ = char.size()
        char = char.view(-1, _)
        word_lens = word_lens.view(-1)
        _char_embedding = self._char_embedding(char.long())
        word_lens[word_lens==0] = 1
        word_lens = word_lens.type(torch.int)
        
        char_embedding, (h_n, cell_n) = self.char_embedding.run(_char_embedding, word_lens)
        char_embedding = h_n.contiguous().view(bs, sl, -1)
        
        word_embedding = self.word_embedding(words.long())
        # concat word embeddings and char embeddings
        word_embedding = torch.cat([word_embedding, char_embedding], dim=-1)
        word_embedding_dp = self.dropout(word_embedding)
        
        seq_lens = seq_lens.type(torch.int)
        out, (n_h, n_cell) = self.rnn.run(word_embedding_dp, seq_lens)
        out_dp = self.dropout(out)
        out = self.linear(out_dp)
        return out.view(out.size(0)*out.size(1), out.size(-1))

In [52]:
ner_model = NER_model(config)

In [53]:
config.ntags = 9

In [54]:
##### rewrite RNN Learner #####
'''rewrite load_encoder to load the encoding modules'''
class RNN_Learner_bidir(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)

    def _get_crit(self, data): return F.cross_entropy
    def fit(self, *args, **kwargs): return super().fit(*args, **kwargs, seq_first=True)

    def save_encoder(self, name_rnn, name_linear): 
        torch.save(self.model.rnn.state_dict(), name_rnn)
        torch.save(self.model.linear.state_dict(), name_linear)
        
    def load_encoder(self, name_rnn, name_linear): 
        self.model.rnn.load_state_dict(torch.load(name_rnn))
        self.model.linear.load_state_dict(torch.load(name_linear))
##### end #####


##### rewrite textmodel #####
'''get layer groups'''
class TextModel_bidir(BasicModel):
    def get_layer_groups(self):
        return [(self.model._char_embedding),(self.model.char_embedding), 
                (self.model.word_embedding),(self.model.rnn),(self.model.linear)]

def freeze_all_but(learner, n):
    c=learner.get_layer_groups()
    for l in c: set_trainable(l, False)
    set_trainable(c[n], True)
    
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
learn = RNN_Learner_bidir(md, TextModel_bidir(to_gpu(ner_model)), opt_fn=opt_fn)
# learn.load_encoder('results/eng_rnn_params.pkl', 'results/eng_linear_params.pkl')

In [65]:
freeze_all_but(learn, -1)
learn.unfreeze()

In [66]:
learn.fit(0.001, 3, metrics=[accuracy], cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                    
    0      0.269262   0.271575   0.97256   
    1      0.191336   0.184959   0.975981                    
    2      0.153102   0.176841   0.975034                    
    3      0.118582   0.125706   0.976686                    
    4      0.077994   0.11118    0.976266                     
    5      0.059355   0.109218   0.976166                     
    6      0.055412   0.109825   0.976551                     



[0.1098248997402647, 0.9765514959048521]

In [69]:
# learn.save_encoder('results/eng_rnn_params.pkl', 'results/eng_linear_params.pkl')

In [63]:
id2tag = [o for i,o in ner_model.idx_to_tag.items()]

In [67]:
eval_ner(learn, id2tag, is_test=True)

Test f1 measure overall: 0.981297056319249
{'precision-PER': 0.9978375538085326, 'recall-PER': 0.9981603153745072, 'f1-measure-PER': 0.997998908495493, 'precision-MISC': 0.280757097791798, 'recall-MISC': 0.2664670658682634, 'f1-measure-MISC': 0.2734254992319008, 'precision-ORG': 0.7019914651493598, 'recall-ORG': 0.7157360406091371, 'f1-measure-ORG': 0.7087971274685316, 'precision-LOC': 0.7793296089385475, 'recall-LOC': 0.7800559179869525, 'f1-measure-LOC': 0.7796925943176026, 'precision-overall': 0.9810436512490914, 'recall-overall': 0.9815505923331611, 'f1-measure-overall': 0.981297056319249}
Test token-level accuracy of NER model: 0.9785.


In [27]:
# padding
import torch.nn.utils.rnn as rnn_utils  

In [43]:
lstm = nn.LSTM(10, 20, bidirectional=True, num_layers=1, batch_first=True)
packed = rnn_utils.pack_padded_sequence(torch.randn(4, 50, 10), T([40, 30, 20, 10]), batch_first=True)
packed_out, packed_hidden = lstm(packed)
unpacked, unpacked_len = rnn_utils.pad_packed_sequence(packed_out)

In [45]:
unpacked, unpacked_len

(tensor([[[-0.0338,  0.0061,  0.0005,  ..., -0.0912,  0.0050,  0.1206],
          [ 0.1199, -0.0857, -0.0399,  ..., -0.0580,  0.1510,  0.1029],
          [-0.0035, -0.0643,  0.0095,  ..., -0.1064,  0.1615, -0.1207],
          [ 0.2136, -0.0328, -0.1314,  ...,  0.0868,  0.0889, -0.0900]],
 
         [[-0.0673, -0.0633, -0.0172,  ..., -0.1604,  0.0258, -0.0048],
          [ 0.0453, -0.2590, -0.1192,  ..., -0.0744,  0.1925, -0.0982],
          [-0.0232, -0.1782, -0.0328,  ..., -0.1592, -0.0142, -0.0713],
          [ 0.1261,  0.0267, -0.0232,  ...,  0.1205,  0.0623, -0.0599]],
 
         [[-0.1563,  0.1211, -0.0770,  ..., -0.2175,  0.1228, -0.0800],
          [-0.0175, -0.2058, -0.0591,  ..., -0.0799,  0.2336, -0.0746],
          [ 0.1256, -0.3455, -0.2181,  ..., -0.1456, -0.0381, -0.0669],
          [-0.0260, -0.0122,  0.0984,  ...,  0.0678,  0.1064, -0.0934]],
 
         ...,
 
         [[-0.1328, -0.0234,  0.0616,  ...,  0.0885,  0.0288, -0.0564],
          [ 0.0000,  0.0000,  0.0000,  

In [57]:
v_rnn = LSTM_v1(10, 20)

In [59]:
out, h = v_rnn.run(torch.ones(4, 50, 10).cuda(), T([10,20,25,25]))